### **Code : 미생물 본설문 - Hierarchical Bayesian**
##### Writer : Donghyeon Kim
##### Update : 2023.11.01.

#### **0. Package Reference**
```
conda create -c conda-forge -n pymc_env "pymc>=5"
conda activate pymc_env

pip install statsmodels

pip install -U scikit-learn
```

#### **1. Prior Settings**

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols # Regression Model
import pymc as pm # Bayesian Analysis
import arviz as az
from sklearn.cluster import KMeans # K-means Clustering

##### Data : 미생물 본설문 OLS

In [2]:
DATA = pd.read_csv('C:/Users/cgpar/Downloads/미생물 HB/미생물_본설문_OLS_Data.csv')
DATA.head(10)

,INDEX,CARD,ORI1,ORI2,ORI3,ENV1,ENV2,CLA1,CLA2,CLA3,EXC1,EXC2,EXC3,NEW1,NEW2,INF1,INF2,INF3,PRICE,Y
0,1,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,150000,7
1,1,2,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,200000,7
2,1,3,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,150000,5
3,1,4,0,0,1,0,1,1,0,0,0,0,1,1,0,1,0,0,200000,8
4,1,5,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,100000,8
5,1,6,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,200000,4
6,1,7,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,100000,8
7,1,8,0,1,0,0,1,0,1,0,0,0,1,1,0,0,1,0,150000,5
8,1,9,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,100000,7
9,1,10,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,100000,7


In [3]:
# PRICE category = (1, 2, 3)
DATA.loc[(DATA['PRICE'] == 100000),'PRICE'] = 1
DATA.loc[(DATA['PRICE'] == 150000),'PRICE'] = 2
DATA.loc[(DATA['PRICE'] == 200000),'PRICE'] = 3
DATA.head(20)

,INDEX,CARD,ORI1,ORI2,ORI3,ENV1,ENV2,CLA1,CLA2,CLA3,EXC1,EXC2,EXC3,NEW1,NEW2,INF1,INF2,INF3,PRICE,Y
0,1,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,2,7
1,1,2,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,3,7
2,1,3,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,2,5
3,1,4,0,0,1,0,1,1,0,0,0,0,1,1,0,1,0,0,3,8
4,1,5,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,1,8
5,1,6,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,3,4
6,1,7,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,8
7,1,8,0,1,0,0,1,0,1,0,0,0,1,1,0,0,1,0,2,5
8,1,9,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,1,7
9,1,10,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,7


#### **2. 전체 응답자 OLS**

In [4]:
def get_ols_params_table(data_df):
    ols_model = ols("Y ~ ORI1 + ORI3 + ENV2 + CLA2 + CLA3 + EXC2 + EXC3 + NEW2 + INF2 + INF3 + PRICE", data=data_df)
    ols_result = ols_model.fit()
    ols_params = pd.DataFrame(ols_result.params, columns=['Coefficient'])
    ols_params['Standard error'] = ols_result.bse
    ols_params['P-value'] = ols_result.pvalues.round(6)
    
    ols_params_2 = ols_params[['Coefficient', 'Standard error']].copy()
    add_params = [
        -ols_params.loc['ORI1',['Coefficient', 'Standard error']] -ols_params.loc['ORI3',['Coefficient', 'Standard error']],
        -ols_params.loc['ENV2',['Coefficient', 'Standard error']],
        -ols_params.loc['CLA2',['Coefficient', 'Standard error']] -ols_params.loc['CLA3',['Coefficient', 'Standard error']],
        -ols_params.loc['EXC2',['Coefficient', 'Standard error']] -ols_params.loc['EXC3',['Coefficient', 'Standard error']],
        -ols_params.loc['NEW2',['Coefficient', 'Standard error']],
        -ols_params.loc['INF2',['Coefficient', 'Standard error']] -ols_params.loc['INF3',['Coefficient', 'Standard error']],
        2*ols_params.loc['PRICE',['Coefficient', 'Standard error']],
        3*ols_params.loc['PRICE',['Coefficient', 'Standard error']]
        ]
    add_params = pd.DataFrame(add_params, columns=['Coefficient', 'Standard error'],
                              index=['ORI2','ENV1','CLA1','EXC1','NEW1','INF1','PRICE_15','PRICE_20'])
    ols_params_2 = pd.concat([ols_params_2, add_params], axis=0)
    ols_params_2.rename(index={'PRICE':'PRICE_10'}, inplace=True)
    ols_params_2 = ols_params_2.loc[['ORI1','ORI2','ORI3',
                                     'ENV1','ENV2',
                                     'CLA1','CLA2','CLA3',
                                     'EXC1','EXC2','EXC3',
                                     'NEW1','NEW2',
                                     'INF1','INF2','INF3',
                                     'PRICE_10','PRICE_15','PRICE_20',
                                     'Intercept'],:]
    
    return ols_params, ols_params_2

In [5]:
total_ols, total_ols_2 = get_ols_params_table(DATA)

# Directory
folder_root = 'C:/Users/cgpar/Downloads/미생물 HB/Result'
if not os.path.isdir(folder_root):
    os.makedirs(folder_root)

# Data Frame -> csv file
total_ols_file_name = folder_root + '/' + '미생물_TOTAL_OLS.csv'
total_ols.to_csv(total_ols_file_name, mode='w')

# Result Check
total_ols

,Coefficient,Standard error,P-value
Intercept,4.246732,0.174057,0.000000
ORI1,0.214869,0.106588,0.043885
ORI3,-0.173203,0.106588,0.104254
ENV2,-0.045343,0.092308,0.623303
CLA2,-0.251634,0.106588,0.018287
CLA3,-1.155229,0.106588,0.000000
EXC2,0.252451,0.106588,0.017913
EXC3,0.698529,0.106588,0.000000
NEW2,1.408088,0.092308,0.000000
INF2,0.816993,0.106588,0.000000


#### **3. 개별 응답자의 부분 가치**

In [6]:
X = DATA[['ORI1', 'ORI3',
           'ENV2',
           'CLA2', 'CLA3',
           'EXC2', 'EXC3',
           'NEW2',
           'INF2', 'INF3',
           'PRICE']].to_numpy().astype(np.float64)
y = DATA['Y'].to_numpy().astype(np.float64)

print(X.shape)
print(y.shape)

(3672, 11)
(3672,)


In [7]:
K = len(DATA['INDEX'].unique()) # 그룹 개수 : 개별 응답자 수
G = np.array([i for i in range(204) for _ in range(18)])

print(K)
print(G)

204
[  0   0   0 ... 203 203 203]


In [8]:
with pm.Model() as HLM:
    
    # Prior probability (proposal distribution) - level2
    mu_a = pm.Normal('mu_1', mu=0, sigma=100)
    sigma_a = pm.HalfCauchy('sigma_1', 5)
    mu_b = pm.Normal('mu_2', mu=0, sigma=100)
    sigma_b = pm.HalfCauchy('sigma_2', 5)

    # Prior probability (proposal distribution)
    a = [pm.Normal(f'theta_Q{i}', mu=mu_a, sigma=sigma_a, shape=K) for i in range(1, X.shape[1]+1)]
    b = pm.Normal('intercept', mu=mu_b, sigma=sigma_b, shape=K)
    eps = pm.HalfCauchy('eps', 5)

    # Model
    y_est = b[G]
    for i in range(len(a)):
        y_est = y_est + a[i][G]*X[:,i]
    likelihood = pm.Normal('y', mu=y_est, sigma=eps, observed=y)

In [9]:
with HLM:
    HLM_trace = pm.sample(1000, progressbar=True, chains=2, cores=12, random_seed=123)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 12 jobs)
NUTS: [mu_1, sigma_1, mu_2, sigma_2, theta_Q1, theta_Q2, theta_Q3, theta_Q4, theta_Q5, theta_Q6, theta_Q7, theta_Q8, theta_Q9, theta_Q10, theta_Q11, intercept, eps]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 60 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [10]:
person_index = DATA['INDEX'].unique()
person_index.sort()

personal_params = az.summary(HLM_trace).iloc[2:-3,[0]]
personal_params['params'] = personal_params.index.str.replace('\[[0-9]{,3}\]', '', regex=True)
personal_params['index'] = [i for _ in personal_params['params'].unique() for i in person_index]

personal_params = personal_params.pivot(columns='params', index='index', values='mean').reset_index()
personal_params.columns = ['INDEX',
                           'Intercept',
                           'ORI1', 'ORI3',
                           'ENV2',
                           'CLA2', 'CLA3',
                           'EXC2', 'EXC3',
                           'NEW2',
                           'INF2', 'INF3',
                           'PRICE']

# Output : csv file
personal_params_file_name = folder_root + '/' + '미생물_HBM_Personal_params.csv'
personal_params.to_csv(personal_params_file_name, mode='w', index=False)

# Result Check
personal_params

,INDEX,Intercept,ORI1,ORI3,ENV2,CLA2,CLA3,EXC2,EXC3,NEW2,INF2,INF3,PRICE
0,1,5.673,0.089,0.699,-0.258,0.427,0.564,-0.614,0.035,1.159,0.161,-0.044,-0.007
1,2,3.701,0.028,0.403,0.263,0.052,-0.267,-0.028,-0.150,0.474,0.051,0.152,0.286
2,5,5.660,0.609,-0.536,-0.060,0.264,-0.025,0.081,0.419,-0.503,0.399,-0.191,0.826
3,11,2.900,0.229,0.159,0.103,0.357,-0.310,0.436,0.112,-0.719,0.518,1.152,0.037
4,16,5.458,0.622,0.232,-0.144,-1.516,0.200,-0.989,-1.093,0.491,1.076,0.034,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,481,3.994,0.500,2.542,-0.103,-0.309,-0.433,0.151,0.037,-0.285,0.053,-0.024,2.769
200,482,2.848,0.181,0.300,0.039,0.846,-0.077,0.224,0.006,-0.092,-0.074,0.101,0.326
201,483,3.481,0.173,0.287,-0.094,0.057,0.005,0.177,0.039,0.182,0.078,0.877,-0.037
202,484,5.297,0.552,0.802,-0.014,-0.142,-0.284,0.191,0.631,-0.064,-0.177,1.294,0.221


#### **4. Clustering**

In [11]:
kmeans = KMeans(n_clusters=2, random_state=456)
clusters = kmeans.fit(personal_params.iloc[:,1:])

personal_params['Cluster'] = clusters.labels_
DATA_cluster = pd.merge(left=DATA,
                        right=personal_params[['INDEX','Cluster']],
                        how='left',
                        on='INDEX')

# Output : csv file
data_cluster_file_name = folder_root + '/' + '미생물_본설문_OLS_Data_Cluster.csv'
DATA_cluster.to_csv(data_cluster_file_name, mode='w', index=False)

# Result Check
DATA_cluster

,INDEX,CARD,ORI1,ORI2,ORI3,ENV1,ENV2,CLA1,CLA2,CLA3,...,EXC2,EXC3,NEW1,NEW2,INF1,INF2,INF3,PRICE,Y,Cluster
0,1,1,0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,1,2,7,0
1,1,2,0,1,0,1,0,1,0,0,...,1,0,1,0,0,1,0,3,7,0
2,1,3,0,0,1,1,0,0,1,0,...,1,0,1,0,1,0,0,2,5,0
3,1,4,0,0,1,0,1,1,0,0,...,0,1,1,0,1,0,0,3,8,0
4,1,5,0,1,0,0,1,0,0,1,...,1,0,0,1,1,0,0,1,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,485,14,1,0,0,1,0,0,0,1,...,0,1,1,0,1,0,0,2,5,1
3668,485,15,0,1,0,1,0,0,1,0,...,0,0,0,1,1,0,0,3,1,1
3669,485,16,0,1,0,1,0,1,0,0,...,0,1,1,0,0,0,1,1,2,1
3670,485,17,1,0,0,0,1,1,0,0,...,1,0,0,1,0,0,1,2,8,1


##### 1st Group

In [12]:
idx_1 = (DATA_cluster['Cluster'] == 0)
group1_ols, group1_ols_2 = get_ols_params_table(DATA_cluster.loc[idx_1,:])
group1_ols

,Coefficient,Standard error,P-value
Intercept,6.612729,0.219763,0.000000
ORI1,0.004854,0.134577,0.971229
ORI3,-0.360841,0.134577,0.007399
ENV2,-0.263754,0.116547,0.023747
CLA2,-0.480583,0.134577,0.000365
CLA3,-1.846278,0.134577,0.000000
EXC2,0.092233,0.134577,0.493205
EXC3,0.425566,0.134577,0.001591
NEW2,0.430421,0.116547,0.000228
INF2,0.783172,0.134577,0.000000


##### 2nd Group

In [13]:
idx_2 = (DATA_cluster['Cluster'] == 1)
group2_ols, group2_ols_2 = get_ols_params_table(DATA_cluster.loc[idx_2,:])
group2_ols

,Coefficient,Standard error,P-value
Intercept,1.833883,0.223870,0.000000
ORI1,0.429043,0.137092,0.001778
ORI3,0.018152,0.137092,0.894678
ENV2,0.177393,0.118725,0.135312
CLA2,-0.018152,0.137092,0.894678
CLA3,-0.450495,0.137092,0.001035
EXC2,0.415842,0.137092,0.002453
EXC3,0.976898,0.137092,0.000000
NEW2,2.405116,0.118725,0.000000
INF2,0.851485,0.137092,0.000000


#### **5. Result Clean-Up**

In [14]:
summary_coef = pd.concat([total_ols_2, group1_ols_2, group2_ols_2], axis=1)
summary_coef.columns = ['Full sample', 'Full sample SE', 'Cluster 1', 'Cluster 1 SE', 'Cluster 2', 'Cluster 2 SE']
add_lines = pd.DataFrame([[len(DATA['INDEX'].unique()),
                           len(DATA_cluster.loc[idx_1,'INDEX'].unique()),
                           len(DATA_cluster.loc[idx_2,'INDEX'].unique())]],
                         columns=['Full sample', 'Cluster 1', 'Cluster 2'], index=['Number of cases'])
summary_coef = pd.concat([summary_coef, add_lines], axis=0)

# Output : csv file
summary_coef_file_name = folder_root + '/' + '미생물_Summary_Coef.csv'
summary_coef.to_csv(summary_coef_file_name, mode='w')

# Result Check
summary_coef

,Full sample,Full sample SE,Cluster 1,Cluster 1 SE,Cluster 2,Cluster 2 SE
ORI1,0.214869,0.106588,0.004854,0.134577,0.429043,0.137092
ORI2,-0.041667,-0.213176,0.355987,-0.269153,-0.447195,-0.274184
ORI3,-0.173203,0.106588,-0.360841,0.134577,0.018152,0.137092
ENV1,0.045343,-0.092308,0.263754,-0.116547,-0.177393,-0.118725
ENV2,-0.045343,0.092308,-0.263754,0.116547,0.177393,0.118725
CLA1,1.406863,-0.213176,2.326861,-0.269153,0.468647,-0.274184
CLA2,-0.251634,0.106588,-0.480583,0.134577,-0.018152,0.137092
CLA3,-1.155229,0.106588,-1.846278,0.134577,-0.450495,0.137092
EXC1,-0.950980,-0.213176,-0.517799,-0.269153,-1.392739,-0.274184
EXC2,0.252451,0.106588,0.092233,0.134577,0.415842,0.137092
